In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'http://www.dicare.org/statistiques/liste-comptes-twitter/'
html = requests.get(url).content

In [3]:
soup = BeautifulSoup(html, "lxml")

In [4]:
trs = soup.find(id='content').find('table').find_all('tr')[1:]
trs

[<tr><td><a href="http://www.dicare.org/lieu/3083-ain/">Ain</a> (1<sup>re</sup>)</td><td><a href="http://www.dicare.org/deputes/427080-xavier-breton/">Xavier <span class="lastname">Breton</span></a></td><td>@<a href="https://twitter.com/bretonxavier">bretonxavier</a></td></tr>,
 <tr><td><a href="http://www.dicare.org/lieu/3083-ain/">Ain</a> (2<sup>e</sup>)</td><td><a href="http://www.dicare.org/deputes/1534217-charles-de-la-verpilliere/">Charles <span class="lastname">de La Verpillière</span></a></td><td>@<a href="https://twitter.com/delaverpilliere">delaverpilliere</a></td></tr>,
 <tr><td><a href="http://www.dicare.org/lieu/3083-ain/">Ain</a> (3<sup>e</sup>)</td><td><a href="http://www.dicare.org/deputes/30334006-olga-givernet/">Olga <span class="lastname">Givernet</span></a></td><td>@<a href="https://twitter.com/OlgaGivernet">OlgaGivernet</a></td></tr>,
 <tr><td><a href="http://www.dicare.org/lieu/3083-ain/">Ain</a> (4<sup>e</sup>)</td><td><a href="http://www.dicare.org/deputes/30337

In [5]:
deputees = [{'name': tr.find_all('td')[1].get_text(), 'twitter': tr.find_all('td')[2].get_text()} for tr in trs]
deputees

[{'name': 'Xavier Breton', 'twitter': '@bretonxavier'},
 {'name': 'Charles de La Verpillière', 'twitter': '@delaverpilliere'},
 {'name': 'Olga Givernet', 'twitter': '@OlgaGivernet'},
 {'name': 'Stéphane Trompille', 'twitter': '@S_Trompille'},
 {'name': 'Damien Abad', 'twitter': '@damienabad'},
 {'name': 'Aude Bono-Vandorme', 'twitter': '@AudeBono'},
 {'name': 'Julien Dive', 'twitter': '@JulienDive'},
 {'name': 'Jean-Louis Bricout', 'twitter': '@jlbricout'},
 {'name': 'Marc Delatte', 'twitter': '@MarcDelatte'},
 {'name': 'Jacques Krabal', 'twitter': '@JKrabal'},
 {'name': 'Jean-Paul Dufrègne', 'twitter': '@JP_Dufregne'},
 {'name': 'Laurence Vanceunebrock-Mialon', 'twitter': '@Laudubray'},
 {'name': 'Bénédicte Peyrol', 'twitter': '@B_Peyrol'},
 {'name': 'Delphine Bagarry', 'twitter': '@DBagarry'},
 {'name': 'Emmanuelle Fontaine-Domeizel', 'twitter': '(aucun)'},
 {'name': 'Éric Ciotti', 'twitter': '@ECiotti'},
 {'name': 'Loïc Dombreval', 'twitter': '@LoicDombreval'},
 {'name': 'Cédric Rou

In [6]:
twitter_accnts = [deputee['twitter'] for deputee in deputees if deputee['twitter'].startswith('@')]

In [7]:
len(twitter_accnts)

520

In [10]:
# filter accounts
import config
import tweepy
api_accnts = []
for api_keys in config.twitter_api_accnts[6:8]:
    auth = tweepy.OAuthHandler(api_keys['consumer_key'], api_keys['consumer_secret'])
    auth.set_access_token(api_keys['access_token'], api_keys['access_token_secret'])
    api_accnts.append(tweepy.API(auth))

In [11]:
api1, api2 = api_accnts

In [12]:
twitter_followers = []
for i, ele in enumerate(twitter_accnts):
    print("Processing user " + ele)
    try:
        user = api1.get_user(screen_name=ele)
    except tweepy.RateLimitError:
        print("\tRate limit error, switching API accounts")
        break
    except tweepy.TweepError as err:
        print("\t{}".format(err))
        pass
    twitter_followers.append([user.followers_count, ele])

while i < len(twitter_accnts):
    ele = twitter_accnts[i]
    print("Processing user " + ele)
    i+=1
    try:
        user = api2.get_user(screen_name=ele)
    except tweepy.TweepError as err:
        print("\t{}".format(err))
        pass
    twitter_followers.append([user.followers_count, ele])

Processing user @bretonxavier
Processing user @delaverpilliere
Processing user @OlgaGivernet
Processing user @S_Trompille
Processing user @damienabad
Processing user @AudeBono
Processing user @JulienDive
Processing user @jlbricout
Processing user @MarcDelatte
Processing user @JKrabal
Processing user @JP_Dufregne
Processing user @Laudubray
	[{'code': 50, 'message': 'User not found.'}]
Processing user @B_Peyrol
Processing user @DBagarry
Processing user @ECiotti
Processing user @LoicDombreval
Processing user @CRoussel_06
	[{'code': 50, 'message': 'User not found.'}]
Processing user @A_Ardisson
Processing user @marinebrenier
Processing user @trastour2017
Processing user @EricPAUGET1
Processing user @MTabarot
Processing user @hsaulignac
Processing user @olivierdussopt
Processing user @FabriceBrun
Processing user @Poletti_B
Processing user @CORDIERPierre08
Processing user @warsmann
Processing user @BenedictTaurine
Processing user @Michel_Larive
Processing user @GBessonMoreau
Processing user 

In [13]:
import pandas as pd
print(len(twitter_followers))
twitter_followers_pd = pd.DataFrame(twitter_followers, columns=['followers', 'twitter'])

521


In [14]:
twitter_followers_pd.head()

,followers,twitter
0,6324,@bretonxavier
1,3521,@delaverpilliere
2,1014,@OlgaGivernet
3,7321,@S_Trompille
4,10260,@damienabad


In [15]:
twitter_followers_pd.describe()

,followers
count,5.210000e+02
mean,1.380996e+04
std,1.170166e+05
min,3.000000e+00
25%,8.990000e+02
50%,1.430000e+03
75%,3.395000e+03
max,1.898029e+06


In [27]:
import scipy.stats
import math
# number of deputees with more than 2000 followers
thr = 5000
q = 1-scipy.stats.percentileofscore(twitter_followers_pd['followers'], thr)/100.0
n = math.ceil(len(twitter_followers) * q)
print("Percentage of deputees with {}+ followers: {}".format(thr, q))
print("Number of deputees with {}+ followers: {}".format(thr, n))

Percentage of deputees with 5000+ followers: 0.16890595009596931
Number of deputees with 5000+ followers: 89


In [28]:
# filter deputees by 2000+ followers
twitter_accnts_final = [ele[1] for ele in twitter_followers if ele[0] > thr]
print(len(twitter_accnts_final)==n)

False


In [29]:
# number of followers
sum_followers = sum([ele[0] for ele in twitter_followers if ele[0] > thr])
sum_followers

6514268

In [31]:
sorted(twitter_followers)

[[3, '@fpetit2017'],
 [4, '@c_vignon3103'],
 [24, '@SPanonacle'],
 [112, '@pascaleboyer05'],
 [118, '@NathalieBassire'],
 [136, '@Justine_BENIN'],
 [141, '@c_naegelen'],
 [148, '@Moetai1'],
 [171, '@fabien_laine'],
 [178, '@Descamps2017'],
 [184, '@AVilliers2017'],
 [193, '@JCGrelier'],
 [196, '@Ericdiard'],
 [204, '@NicoleTrisse'],
 [230, '@AdrienMorenas'],
 [248, '@Leclerc2017'],
 [249, '@michelfanget'],
 [251, '@BrunoDuverge'],
 [271, '@lejeune_ch'],
 [283, '@AudeLuquet2017'],
 [288, '@isabelle_rauch'],
 [292, '@SophieMette1'],
 [295, '@trastour2017'],
 [313, '@FranceLorho'],
 [317, '@CKamowski3805'],
 [332, '@mtamverhaeghe'],
 [337, '@HeleneVainqueur'],
 [347, '@David_Lorion'],
 [350, '@fgranjus'],
 [350, '@fgranjus'],
 [355, '@eanthoine26'],
 [366, '@VBazinMalgras'],
 [376, '@VRolland73'],
 [380, '@Cecile_Rilhac'],
 [389, '@JJ_Gaultier'],
 [406, '@2017alaindavid'],
 [410, '@RaphaelGerard17'],
 [418, '@rscactu'],
 [421, '@JhRatenon'],
 [424, '@TuffnellLREM17'],
 [432, '@DelponMiche

In [39]:
# get twitters of french organizations
import os
data_dir = os.path.join('..', 'data')
# file with twitter of relevant french organizations
fr_twitter = os.path.join(data_dir, 'twitter-france.txt')
with open(fr_twitter, 'r') as fin:
    france_orgs_twitter = [line.strip() for line in fin if line.strip().startswith('@')]
france_orgs_twitter

['@lemondefr',
 '@Le_Figaro',
 '@LEXPRESS',
 '@Rue89',
 '@libe',
 '@20Minutes',
 '@lobs',
 '@Slatefr',
 '@mediapart',
 '@LePoint',
 '@LesEchos',
 '@le_Parisien',
 '@atlantico_fr',
 '@FN_officiel',
 '@lesRepublicains',
 '@UDI_off',
 '@enmarchefr',
 '@partisocialiste',
 '@EELV',
 '@FranceInsoumise',
 '@PCF']

In [41]:
twitter_accnts_final += france_orgs_twitter
len(twitter_accnts_final)

208

In [43]:
import utils
f_twitter = os.path.join(data_dir, 'fr_elites_handles.json')
f_deputees = os.path.join(data_dir, 'fr_elites_twitter_map.json')

utils.json_dump(twitter_accnts_final, f_twitter)

#remove deputees with twitter account
deputees_twitter_map = [ele for ele in deputees if ele['twitter'].startswith('@')]
print(len(deputees_twitter_map))
utils.json_dump(deputees_twitter_map, f_deputees)

520
